# Identify a video for the March PI Meeting Demo

In [1]:
import os
import json
import simplejson

import pandas as pd

from charm.data import utils

In [2]:
meta_filepath = '/home/iron-man/Documents/data/charm/transformed/metadata.csv'
meta_df = pd.read_csv(meta_filepath)

In [3]:
meta_df

/home/iron-man/Documents/charm/.venv/lib/python3.10/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/home/iron-man/Documents/charm/.venv/lib/python3.10/site-packages/IPython/core/formatters.py:342: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,release,catalog_id,file_uid,url,modality,start,end,transcribed,utterance_count,emotion_count,...,unwrapped_md5,download_date,content_date,status_in_corpus,legacy_catalog_id,original_file_id,type,file_path,length,version
0,R1,LDC2022E11,M010009A4,na,video,NaN,NaN,True,45.0,NaN,...,1c797fd8bd832fe4c7244a7b9b0aa2a7,na,na,present,LDC2015R18,VVC008300,NaN,NaN,NaN,V1.0
1,R1,LDC2022E11,M010009BC,na,video,0.0,300.0,True,90.0,3.0,...,4b121a497725dcf1f232d00c119bd823,na,na,present,LDC2015R18,VVC016445,NaN,NaN,NaN,V1.0
2,R1,LDC2022E11,M010009BE,na,video,NaN,NaN,True,67.0,NaN,...,83d42079d22abe1f576be92afc182474,na,na,present,LDC2015R18,VVC020973,NaN,NaN,NaN,V1.0
3,R1,LDC2022E11,M010009CZ,na,video,NaN,NaN,True,89.0,NaN,...,07843653514dad5894b5782fe1475ee4,na,na,present,LDC2015R18,VVC011554,NaN,NaN,NaN,V1.0
4,R1,LDC2022E11,M010009D0,na,video,NaN,NaN,True,29.0,NaN,...,f0a8d0563a8d6480675f21df6c0acbd2,na,na,present,LDC2015R18,VVC011561,NaN,NaN,NaN,V1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16193,Mini-Eval,LDC2022E22,M01000HUA,na,text,0.0,678.0,NaN,NaN,NaN,...,72c4da8e36c41fc1eef2e0ebcbd19da3,na,na,present,na,na,text,data/text/psm/M01000HUA.psm.xml,681.0,V1.0
16194,Mini-Eval,LDC2022E22,M01000HUF,na,text,NaN,NaN,NaN,NaN,NaN,...,c2e8be27766da87f47d1a234e64bb7eb,na,na,present,na,na,text,data/text/psm/M01000HUF.psm.xml,982.0,V1.0
16195,Mini-Eval,LDC2022E22,M01000HUK,na,text,NaN,NaN,NaN,NaN,NaN,...,f226937cf10a0233c14bd3142f561c86,na,na,present,na,na,text,data/text/psm/M01000HUK.psm.xml,1224.0,V1.0
16196,Mini-Eval,LDC2022E22,M01000HUP,na,text,517.0,1129.0,NaN,NaN,NaN,...,33b9d19215c93ca6983dc1b6d2800754,na,na,present,na,na,text,data/text/psm/M01000HUP.psm.xml,1688.0,V1.0


In [4]:
# subset to mini-eval videos
file_filter = (meta_df['release'] == 'Mini-Eval') & (meta_df['modality'] == 'video')
df = meta_df[file_filter]

In [5]:
# find a file that's been annotated with all tasks
df = df[df[['valence_arousal_count', 'norms_count', 'changepoint_count', 'emotions_count']].notnull().all(axis=1)]

In [6]:
# TODO: load annotations and find a video with a reasonable number of annotations

In [7]:
# 7
video = df.sort_values(by=['utterance_count'], ascending=[True]).iloc[8]

In [8]:
video['file_uid']

'M010040RH'

In [9]:
with open(f'{video["file_uid"]}_metadata.json', 'w') as f:
    simplejson.dump(video.to_dict(), f, ignore_nan=True)

In [10]:
# strip LDC header
# translate with whisper

In [11]:
# load annotations for this file
result = utils.load_ldc_annotation('/home/iron-man/Documents/data/charm/raw/LDC2023E01_CCU_TA1_Mandarin_Chinese_Mini_Evaluation_Annotation_Unsequestered/')

In [12]:
anno_dfs, segment_df, version_df = result

In [13]:
anno_dfs.keys()

dict_keys(['valence_arousal.tab', 'changepoint.tab', 'norms.tab', 'emotions.tab'])

## Valence and Arousal

In [14]:
valence_df = anno_dfs['valence_arousal.tab']

# filter out noann rows
valence_df = valence_df[valence_df['valence_continuous'] != 'noann'].reset_index(drop=True)

# convert cols to floats
valence_convert_cols = ['valence_continuous', 'valence_binned', 'arousal_continuous', 'arousal_binned']
valence_df.loc[:, valence_convert_cols] = valence_df[valence_convert_cols].astype(float)

# drop the user_id column
valence_df.drop(columns=['user_id'], inplace=True)

# average valence over annotators
valence_df = valence_df.groupby(['file_id', 'segment_id'], as_index=False).mean()

valence_df = valence_df.merge(segment_df.drop(columns=['file_id']), how='left', on='segment_id')

valence_df = valence_df[valence_df['file_id'] == video['file_uid']]

valence_df

## Emotions

In [22]:
emotions_df = anno_dfs['emotions.tab']

# drop emotion == 'none'
emotions_df = emotions_df[(emotions_df['emotion'] != 'none') & (emotions_df['emotion'] != 'noann')]

# split emotion string and explode df
emotions_df['emotion'] = emotions_df['emotion'].apply(lambda x: x.split(','))
emotions_df = emotions_df.explode(column='emotion')

# drop user_id
emotions_df.drop(columns=['user_id', 'multi_speaker'], inplace=True)

# group by file_id, segment_id, create a set of emotions
emotions_df = emotions_df.groupby(['file_id', 'segment_id'], as_index=False).agg(set)

# convert set to ordered list
emotions_df['emotion'] = emotions_df['emotion'].apply(lambda x: ', '.join(sorted(list(x))))

emotions_df = emotions_df.merge(segment_df.drop(columns=['file_id']), how='left', on='segment_id')

emotions_df = emotions_df[emotions_df['file_id'] == video['file_uid']]

emotions_df

## Norms

In [31]:
norms_df = anno_dfs['norms.tab']

# norms map
norms_map = {101: 'doing apology',
102: 'doing criticism',
103: 'doing greeting',
104: 'doing request',
105: 'doing persuasion',
106: 'doing thanks',
107: 'doing taking leave'}

norms_df = norms_df[(norms_df['norm'] != 'none') & (norms_df['norm'] != 'noann')]

norms_df.drop(columns='user_id', inplace=True)

norms_df = norms_df.groupby(['file_id', 'segment_id'], as_index=False).agg(list)

norms_df = norms_df.merge(segment_df.drop(columns=['file_id']), how='left', on='segment_id')

norms_df = norms_df[norms_df['file_id'] == video['file_uid']]

# pull in norms map
norms_df['norm'] = norms_df['norm'].apply(lambda x: [norms_map[int(y)] for y in x])

norms_df

## Changepoint

In [40]:
changepoint_df = anno_dfs['changepoint.tab']

changepoint_df.drop(columns='user_id', inplace=True)

changepoint_df = changepoint_df[changepoint_df['file_id'] == video['file_uid']]

changepoint_df

## Create JSON file containing all annotations

In [44]:
video_annos = {}
video_annos['valence_arousal'] = valence_df.to_dict('records')
video_annos['emotion'] = emotions_df.to_dict('records')
video_annos['norms'] = norms_df.to_dict('records')
video_annos['changepoint'] = changepoint_df.to_dict('records')

video_annos['changepoint']

with open(f'{video["file_uid"]}_annotations.json', 'w') as f:
    simplejson.dump(video_annos, f)

### Retrieve predictions for video

In [47]:
cd = 'CCU_P1_TA1_CD_COL_LDC2022E22-V1_20221128_150559'
ad = 'CCU_P1_TA1_AD_COL_LDC2022E22-V1_20221123_141038'
vd = 'CCU_P1_TA1_VD_COL_LDC2022E22-V1_20221123_144304'
ed = 'CCU_P1_TA1_ED_COL_LDC2022E22-V1_20221128_150401'
nd = 'CCU_P1_TA1_ND_COL_LDC2022E22-V1_20221129_101234'
# load predictions for chosen file
submissions_dir = '/home/iron-man/Documents/data/charm/transformed/predictions/submissions'
prediction_dfs = {}
for pred_dir, pred_type in [(cd, 'cd'), (ad, 'ad'), (vd, 'vd'), (ed, 'ed'), (nd, 'nd')]:
    pred_file = os.path.join(submissions_dir, pred_dir, f"{video['file_uid']}.tab")
    prediction_dfs[pred_type] = pd.read_csv(pred_file, delimiter='\t')

In [48]:
# merge valence and arousal
# NB: this won't always work if timestamps aren't aligned
vd_ad_preds_df = prediction_dfs['ad'].merge(prediction_dfs['vd'], how='inner', on=['file_id', 'start', 'end'])

# group ['file_id', 'start', 'end'] and aggregrate to a list
nd_preds_df = prediction_dfs['nd'].groupby(['file_id', 'start', 'end'], as_index=False).agg(list)

temp_df = prediction_dfs['ed'].groupby(['file_id', 'start', 'end'], as_index=False).agg(list)

# check if any intervals have multiple emotions
(temp_df['emotion'].apply(lambda x: len(x)) > 1).sum()

# pull in norms map
nd_preds_df['norm'] = nd_preds_df['norm'].apply(lambda x: [norms_map[int(y)] for y in x])

video_preds = {}
video_preds['valence_arousal'] = vd_ad_preds_df.to_dict('records')
video_preds['emotion'] = prediction_dfs['ed'].to_dict('records')
video_preds['norms'] = nd_preds_df.to_dict('records')
video_preds['changepoint'] = prediction_dfs['cd'].to_dict('records')

with open(f'{video["file_uid"]}_predictions.json', 'w') as f:
    simplejson.dump(video_preds, f)